### First part: recover the encrypted messages from the unprotected channel

In [37]:
messages = """
[23:30] <Doomfang> : e4f127f0389fd25b6a69cc723f50
[23:32] <Stormbane> : e4f127f0389fc5406a76c77130593f
[23:34] <Runeblight> : e4f127f0389fc4416b61c87f38503267
[00:00] <Stormbane> : 92fa69e5369ff15b7124cb333f522d3358eb1af978c8decf37d27228e968d490dfeeea9e56d8d7041f6c41c19b8dfe3284c969e5b4a9e14713214067e28b17fe428d
[00:01] <Doomfang> : 90f12af621cce25b6a6084331956293358ea0fab7c8ad3df269c2621e2318686d4ece8cd179ebb15032e5fc1828dfc30cbd36cfefaada9550e6f5370ab9203f807cf24421bd1d327d3f5a5
[00:02] <Runeblight> : 8bf03ab32adae2182566df67717e7d7e0ce102bc7ac1d8d424d2752fe12d8680d3fef5981994bb12022c5c809c9bb562edc03ce3fcaff00614645e66eedd03f90b8332544f9cd72fcde4baab4db29a20129d48a7104a238273e63ab2336de3d1aca4d7379f2f
[00:04] <Doomfang> : 91f727e073dcfe556b6acf7f715e293342ed1ef96acbd7df63946932ac24c99bdaabf28c1493e84f4b075795d79bbb31d3cf68f4fceafd49476e4567ab8d04e351c231544f83d127c8be
[00:04] <Doomfang> : 8dfa3cf673d6e514716ccf33215629605cea18b86acf91dc2c80262ff93a8686d8e8f39f1dd8f8090a255c849cd2bb67c9cf2ef0e282c1653134564ae09e14b77d9733642380d111efb6f591
[00:05] <Runeblight> : 82f03ab33acbb8144a6ac66a714432725ee74ab06d8ac6d3379a262ff93a8698d2f8f2cd0c8aee121f2e56c19184f72bc1d532
[00:06] <Stormbane> : 9cfa3dbd73f0e3462568cb602517377c5ae74ab478d391d222846360e02dc0819dfff48c1b9de84f4b1c57c19d9de83684c479b7e2affb5f47625167ee9b03e609
[00:07] <Doomfang> : 8cb823b330d7f3576e6dc47471582f610cee05be6a8ac5d563906360ff3dd4909de5e9cd0c8afa020e6b5d87d087ee3084c77fe3fda5e75547735578ea9418f909
[00:08] <Runeblight> : 8efa2be373d2f3147074ce7225523e3d0ccb0cf96dc2d4c363916734ef20869ad3a7a69a1ddff70d4b23539795c8ef2d84c77fe3b4ace855132f
[00:09] <Stormbane> : 8cb822ff73dcf9597565d876714332760cee0bad7cd9c59a27937221ac3fcf81d5abe9980ad8f90008204791d098f723ca883cc0f1eae45314751070f99c05ef07c22b484f82d72fcbb0eda60cb39a7203d200a10a4e63
[00:10] <Doomfang> : 8cf96ef625dae44d716cc37d361733600ce106bc78d89d9a3497262de33ec3d5c9e4a699109dbb0f0e3346c1839cfa25c1883cd8e1b8a94108605c35e28e56fd4ed72d5801d1cc2dc4f3f2ed
[00:11] <Runeblight> : 8df022f773d0f81a254d8d7e71443f7645ec0df96adec3db2d956360ff21c19bdce7f5cd1e8af40c4b2447958381ff278a864bf2b4a7e0410f751077eedd01eb53c02d540bdf
[00:12] <Stormbane> : 92fa6ef032d1b1402570cb7834173b7d55a218b06ac1c29463be6334ab3b8699d8eaf088588cf308186b51899186f527c8867ef2f2a5fb4347755870f2dd02f846c02e111a8290
[00:13] <Doomfang> : 84f83cf636dbb814486bdc767156367f0cf60bb572d991ce2cd27228e968d687d4fde7991dd8e90e04261cc1a29df527c6ca75f0fcbea506176d5574f89856e94bc624434f85d62d85fcf5a45fe49765149746
[00:14] <Runeblight> : 90f12af621cce25b6a6084331810373348eb19ba76c4dfdf20866f2eeb68c89acaa5a6a41ed8ef090e321289919efe62d7c379f9b4bffa0a47765535e68805fe07c72c420e81ce2dc4e2baaa41a99a640f931ca1145263
[00:53] <Doomfang> : e4f32bf225da
[00:53] <Stormbane> : e4f32bf225da
[00:53] <Runeblight> : e4f32bf225da
"""


In [38]:
import re
from datetime import datetime

def parse_messages(messages):
  # Define a pattern to match the messages
  pattern = re.compile(r'\[(.*?)\] <(.*?)> : (.*)')

  # Initialize an empty list to store the parsed messages
  parsed_messages = []

  # Iterate over each line in the messages string
  for line in messages.strip().split('\n'):
    match = pattern.match(line)
    if match:
      time_str, name, body = match.groups()
      time = datetime.strptime(time_str, '%H:%M')
      # Divide the body into a list of bytes long 256 bits (64 hex characters)
      parsed_messages.append({'datetime': time, 'name': name, 'body': bytes.fromhex(body)})

  return parsed_messages

def xor(a, b):
  return bytes(x ^ y for x, y in zip(a, b))

In [39]:
parsed_messages = parse_messages(messages)
body_lengths = [len(msg['body']) for msg in parsed_messages]
print(body_lengths)

[14, 15, 16, 66, 75, 102, 74, 76, 51, 65, 65, 58, 87, 76, 70, 71, 83, 87, 6, 6, 6]


In [40]:
def pad(msg):
  # pad the message to a multiple of 16 bytes
  return msg + b'\x00' * (16 - len(msg) % 16)

In [41]:
hyp_message = b"Understood. I'm disconnecting now. If they have seen us, we must disappear immediately"
hyp_message = pad(hyp_message)

doomfang_cipher = parsed_messages[17]['body']

e_k = xor(hyp_message, doomfang_cipher)
print("E_k recovered: ", len(e_k), e_k)

for i, msg in enumerate(parsed_messages):
  message = xor(e_k, msg['body']).decode()
  
  print(f"{i:2} <{msg['name']:<10}> {message}")

E_k recovered:  87 b"\xc5\x9fN\x93S\xbf\x964\x05\x04\xaa\x13Q7Z\x13,\x82j\xd9\x19\xaa\xb1\xbaC\xf2\x06@\x8cH\xa6\xf5\xbd\x8b\x86\xedx\xf8\x9bakK2\xe1\xf0\xe8\x9bB\xa4\xa6\x1c\x97\x94\xca\x89&g\x010\x15\x8b\xfdv\x8a'\xa3E1o\xf1\xbeH\xa5\x90\x9a\xc3,\xc4\xff\x00f\xf2h\xc4x+c"
 0 <Doomfang  > !nick Doomfang
 1 <Stormbane > !nick Stormbane
 2 <Runeblight> !nick Runeblight
 3 <Stormbane > We've got a new tip about the rebels. Let's keep our chat private.
 4 <Doomfang  > Understood. Has there been any sign of them regrouping since our last move?
 5 <Runeblight> Not yet, but I'm checking some unusual signals. If they sense us, we might have to cha@
 6 <Doomfang  > This channel is not safe for long talks. Let's switch to our private room.
 7 <Doomfang  > Here is the passphrase for our secure channel: %mi2gvHHCV5f_kcb=Z4vULqoYJ&oR
 8 <Runeblight> Got it. Only share it with our most trusted allies.
 9 <Stormbane > Yes. Our last move may have left traces. We must be very careful.
10 <Doomfang  > 

### Second part: recover the messages in the secret forum

In [42]:
secret_messages = """
[00:10] <Doomfang> : e4f127f0389fd25b6a69cc723f50
[00:12] <Stormbane> : e4f127f0389fc5406a76c77130593f
[00:14] <Runeblight> : e4f127f0389fc4416b61c87f38503267
[00:26] <Runeblight> : 92fa6ee03bd0e3586124c17634477a7c59f04aa975cbdfd42a9c6160e42dd49093abd2851dd8f4141f2e40c19889f72ed7867de5f1eae74913214370e88804ef0b83245f0bd1ca27cab0f7a242bddf651f971be40f4a39867ee621af246df8c0aabe9e3784602ed2861d0cd1
[01:01] <Doomfang> : 84f83cf636dbb814516ccf3334593f7e55a519f96ac9decf37812627fe27d1d5d0e4f4885888fe13182241959586ef6c84ef7ab7e0a2ec5f47625161e89556ef51c62b110ed1c920cce3eaa65ee49066469d1db6584f28967fa13bb46d6de3d8aaa99e23856d2c9c8e1e099acdb457265383f0dc7abe80da152548594bc4724bcebf423a33b9d71081ce3c0ffa1b666115dc6ae0d41a4629043fbf720b551abccd5907b5943bf057294d0fec0d54e534bdaa71
[01:02] <Runeblight> : 8cb838f673ddf3516b24d9672453237a42e54aad71cf91ce31936525ff68ca90dbffa68f1d90f20f0f6b5098d087ee3084d66ef2e2a3e6531421597be89c18fe46d72c5e01829268c4fefee35fab9265129a01aa1f0b2b8073aa26e7363ff8dea8fe9e1b997360d2860508909fbe10285ccdf0d83fa79f87151a4c0a06c264519af1423b67f8c81589d77d17e15e7b7c15ce61a5d3070d281968a222005508ffd6580aae883fb34861430ebe4348ff34edb6396946214699e1604421dfe5f6a50be29fb9197edd48c2a6b044b719a595e9e75c092f9e952614ee8571ba3a02a80e36de09aaea8345f1346f9f414deaf51f97c0de66f8f7054227861ff6578acb16c3ca5621d4f2499cd06e261569ee1f9490fd68b947a6abd5467870920c33d16067f2ab8775d25916face74569f614c07889cd0cbcc5e91dfc0bc8044ae551fddbe7fdd4189b5ac81305f10f97d772574f612c764dfbd9b526f
[01:03] <Stormbane> : 8cb823b332d3e4516460d333324535605faf09b17cc9dad32d95262ff93a8686cdeeea810f97e90a4b2a55809986e83684d274f2b4abe7450e645e61ab8f13e948d1214241d1f72e85e4f2aa5fe49d65079107aa585c2c9636b634b5356df8d6efb1d074836d24d991511a918ebd51294e80e6c62ee7d3bd121e415940d86f5bcea15f217cbe955caccc685bfb182f7a418f66f6860805321f69a9220c5f1eb099424fa4887ae459294a1be84807f071bdab302b5f370bd6b45d586ecee3f9ac0becdd8851768e0bcea7a64db255a294e9e55a0e3392952856a9ca74fb21126aedcdc203eeb98e0cf13a3a850859e2f11fc497d275e5f71b4132c518f75789c116dfda4773d4e80c90ca2b225e69d2168d80ba79a21c
[01:04] <Doomfang> : 92fa6ef032d1f85b7124cb75375828770cea0faa70ded0ce2a9d686eac01c0d5c9e3ef9e5891e8410a6b50939589f82a888668fff1a4a9520f64105de29a1eaa64cc305f0c98d26fd6b0fcac5ea79a73469f09bd584a219773a731be612ff290a0be9e3b997360c891101693c3f575315f83a3dc32aed3875813411543c2751f83b85e3a72b3de5c8dd66917f65e6b7c5ac22fead31b4623186ba57007100afdd44646a88134bd1c5e475af35854e571aeb6312f5a200bd8f5600c21c3efe9e006aa9f8204658e09d3aaa446b619a395afec5d0e2284db2959b88f23f83613e64d3ac408beb88e4db8262a9306
[01:05] <Runeblight> : 80e72ff027d3ef1a2545c47771522c7642a203bf39ddd49a31976b21e5268680d3f8e38816d8fd0e196b5c8e87c4bb35c18672f2f1aea945086f447ce59a13e444da65410390d03b8bb0d3a50cb0976546b107b11648248936a03ab53524f1d9aaa39e3d987260d18216169c8cb910255b9ff1c13fb980d81505485945de74538af1412160bd9b1d8dda7908e15e7b7c15db67e0cf1b4635026da36c055806f0dd4509e1a235b34b6c0212ff5b42b130edaa3a2a5c3c0299e66d0c23c8ede2b34fa3d9cd1479da1ad8e9ac4ef34daf95e9f941142a96893818a98b77ff731ffb4d2ace04a2af851f
[01:06] <Stormbane> : 9cfa3dbf73dde3402573cf333c4229670cf618bc78de91d337d2692ee0318694ceabe7cd1499e8154b3957929f9aef6c84ef7ab7e3afa94704755963ea8913aa4ed76545009e9e3bcafff4ef0cb39a20149b1baf5859289373a739ae2f2ab7d9bba39e38836221c88a1e11d1cd9c4467539ea3c43ba9969850160d18558b2177ba93560d61b1d923aacb7d1cf51761746aea77f5ca060f32176ba56d0c6f3ef5cd5e788a8323cc72664c19fb7275f424bebc7e34
[01:07] <Doomfang> : 82f021f77d9fd85b2576cf703e453e3343e44ab06d8adccf30862625f421d5819de2e8cd0c90fe411c395b95848df562d0c971f2e7e4a96f47765979e7dd13e454d637544f90d22485e4e8a24fa18c2007800de41d592c9673a279e72023f390a6a49e2784602cd0c31f1a8988a710255fcdf0d835a0969a151d4b5949c1645182a8036e5abe9b0886dc3c1efc1b626a15ca79e0d4490a23176da271425f0fbcd0420be1913fb34b604e16be4546e734edb7306940370597fa700c2dc5ede2a30ae2
[01:08] <Runeblight> : 84f83cf636dbb814516ccf333c5828760cf50ff97dc3c2d936817560e53c8ad5c9e3e3cd1f8afe001f2e40c18480fe62d6cf6ffcbaeacc5002734935e6921bef49d765460ad1da2dc9f1e3ef0cb0976546b107b11648248936b521b52423f0c4a7b5d027cc6834cfc3151a9988bb432249c3a3ff3feb9e8146060d1845c5214c81be436e71bddd139cdc3c14e70c2f645cc16bead14909205670bc720d421de9d75f53b8c639ff537a4709b0
[01:09] <Stormbane> : 92fa6ee03bd0e3586124cf7d35172e7b45f14ab47ccfc5d32d952621e22c8698d2fde3cd0c97bb004b265d9395c8e827c7d36ef2b4b9e84804754578a5dd3fec07d72d5406839e25c4f7ffb00cab8d20158201a10b0b2c9773e636ab2e3efedea8f0d73ac02134d486085f928cac102e5499e6da39ae8380151d580b06c66e4d8aa2036e44bd9b119bca685bfc117b3341ce64e0861d0e27023faf6a035e0af997166ba4927ae75460515afc4807e539a8f9332840264695f1675f2fcae9aca901eccb8518648e18cda8a64dfd
[01:09] <Runeblight> : e4f32bf225da
[01:09] <Stormbane> : e4f32bf225da
[01:09] <Doomfang> : e4f32bf225da
"""

secret_parsed_messages = parse_messages(secret_messages)

In [43]:
hyp_message = b"Good. No record of it must exist in the written tomes. I will ensure all traces are erased, and it shall never be spoken of openly. If the enemy ever learns of it, we will have no "
hyp_message = pad(hyp_message)

cipher = secret_parsed_messages[10]['body']

e_k = xor(hyp_message, cipher)
print("E_k recovered: ", len(e_k), e_k)

for i, msg in enumerate(secret_parsed_messages):
  message = xor(e_k, msg['body']).decode()
  
  print(f"{i:2} <{msg['name']:<10}> {message}")

E_k recovered:  192 b'\xc5\x9fN\x93S\xbf\x964\x05\x04\xaa\x13Q7Z\x13,\x82j\xd9\x19\xaa\xb1\xbaC\xf2\x06@\x8cH\xa6\xf5\xbd\x8b\x86\xedx\xf8\x9bakK2\xe1\xf0\xe8\x9bB\xa4\xa6\x1c\x97\x94\xca\x89&g\x010\x15\x8b\xfdv\x8a\'\xa3E1o\xf1\xbeH\xa5\x90\x9a\xc3,\xc4\xff\x00f\xf2h\xc4x+M\xe5\x16\xc6U\xc7AM\x97\xb0\xcf\xd0\xbeT\xec\x01@\xbc\xe3q\x7f\xff\xed\xd50G:\xed\x83\xa8Z\xcb\xf3\xf45r-y&\xb1\x01?\xee\xd1-N\x13\xd8\xbb|\xee\xb9\x1c{\x92~\x0f\x135\xaf\x0f\x85\xa6ifFv\x1f\xcc\x02b0i\x9c\xb96\'\xc1\xe6Z\x93<\t"z\x9e-\'\x91Q\xcd\xd9_I@7\x05\x97\xfap\x0c-\xc5\xed\xe2\xa3'
 0 <Doomfang  > !nick Doomfang
 1 <Stormbane > !nick Stormbane
 2 <Runeblight> !nick Runeblight
 3 <Runeblight> We should keep our planning here. The outer halls are not secure, and too many eyes watch the open channels.
 4 <Doomfang  > Agreed. The enemy's scouts grow more persistent. If they catch even a whisper of our designs, they will move against us. We must not allow their seers or spies to track our steps.
 5 <Runeblight> I'

So the flag is: HTB{Crib_Dragging_Exploitation_With_Key_Nonce_Reuse!}